In [1]:
import csv
import collections
import feather
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from scipy import sparse
import numpy as np

In [2]:
import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from data.basic_functions import * 

# Inspect file (SQL version)

In [3]:
data_path = '../data/raw/'
holdings = load_data(data_path,'holdings')

In [129]:
holdings.sample()

,crsp_portno,report_dt,percent_tna,nbr_shares,market_val,crsp_company_key,calc_tna_x,calc_tna_y,calc_percent_tna
1836572,1001174,2005-04-30,NaN,10266.0,535269.24,3010769,NaN,NaN,NaN


In [118]:
portfolio_sum = holdings[['crsp_portno','report_dt','percent_tna']].groupby(['crsp_portno','report_dt']).agg('sum')

In [123]:
portfolio_sum_sorted = portfolio_sum.sort_values(by='percent_tna')

In [127]:
portfolio_sum_sorted[-10:]

,,percent_tna
crsp_portno,report_dt,
1002320,2005-09-30,493.620771
1001817,2005-03-31,495.621773
1001936,2005-03-31,498.017963
1002723,2005-06-30,675.461594
1002183,2004-09-30,946.706361
1002033,2003-06-30,999.102318
1002200,2003-06-30,1677.112366
1001742,2005-09-30,2038.701793
1001342,2004-03-31,7554.365732


In [152]:
mask = (holdings['crsp_portno'] == 1002320) & (holdings['report_dt'] == '2005-09-30')
holdings[mask ]

,crsp_portno,report_dt,percent_tna,nbr_shares,market_val,crsp_company_key,calc_tna_x,calc_tna_y,calc_percent_tna
7151144,1002320,2005-09-30,36.198242,29624.0,1882308.96,3003728,52000.010118,52000.017486,36.198242
7151145,1002320,2005-09-30,31.929520,49312.0,1660335.04,3004314,52000.000565,52000.017486,31.929520
7151146,1002320,2005-09-30,23.125320,46736.0,1202517.28,3006728,52000.026698,52000.017486,23.125320
7151147,1002320,2005-09-30,16.793533,13800.0,873264.00,3005747,52000.015904,52000.017486,16.793533
7151148,1002320,2005-09-30,13.606796,28704.0,707553.60,3005322,52000.014275,52000.017486,13.606796
7151149,1002320,2005-09-30,13.562637,9568.0,705257.28,3000504,52000.010240,52000.017486,13.562637
7151150,1002320,2005-09-30,13.044609,11408.0,678319.68,3008440,52000.000642,52000.017486,13.044609
7151151,1002320,2005-09-30,13.024651,15456.0,677281.92,3011176,52000.006923,52000.017486,13.024651
7151152,1002320,2005-09-30,11.638069,7544.0,605179.68,3005388,52000.007222,52000.017486,11.638069
7151153,1002320,2005-09-30,10.208885,29624.0,530862.08,3002250,52000.004895,52000.017486,10.208885
